In [1]:
import json
from objectbox import Entity, Id, String, Store, Box, Float32Vector, HnswIndex, VectorDistanceType
from pydantic import BaseModel
from sentence_transformers import SentenceTransformer
from tqdm import tqdm


/home/artem.d/Projects/Python/tabletopmagnat/.venv/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
rules = [{"id": "intro_001", "content": "ПОДЗЕМЕЛЬЯ ПЁСИКИ\nПравила игры\n2–4 игрока\nОт 8 лет\n30 минут\nВ волшебном королевстве каждый год проходят соревнования по побегу из подземелья. Самые хорошие и смелые пёсики спускаются в глубокие мрачные комнаты, чтобы сразиться с чудовищами и собрать как можно больше трофеев. Победителей ждут почести и слава, титулы и горы золота, а также очень вкусные косточки.", "metadata": {"section": "Введение", "subsection": "Общая информация"}},
{"id": "goal_002", "content": "В этой игре вы станете участниками фэнтезийных соревнований. В каждом раунде вы по очереди выкладываете карты комнат, формируя подземные лабиринты с монстрами и сокровищами.\nЗадача каждого игрока — построить наиболее выгодный для себя маршрут к выходу из подземелья, по пути собирая трофеи, монеты и побеждая чудовищ. Победит тот, кто по итогам трёх раундов накопит больше всего золотых монет — их можно вынести из подземелья, получить за выполненные квесты и за побеждённых монстров.", "metadata": {"section": "Цель игры", "subsection": ""}},
{"id": "components_003", "content": "Состав\n90 карт комнат\n6 планшетов пёсиков\n12 карт квестов\n6 карт зелий\n42 жетона лапок\n(по 7 жетонов каждого цвета)\n3 жетона раундов\nЖетон первого игрока\nЭти правила игры", "metadata": {"section": "Компоненты", "subsection": "Список компонентов"}},
{"id": "setup_004", "content": "Подготовка\nКаждый игрок выбирает себе планшет пёсика и кладёт перед собой лицевой стороной вверх. Лишние планшеты уберите в коробку.\nЗатем каждый игрок берёт себе 7 жетонов лапок, указанных в углу планшета: 5 из них кладёт на планшет (это базовые жетоны), 2 оставшихся кладёт рядом с планшетом (это дополнительные жетоны). Остальные жетоны уберите в коробку.\nПеремешайте все карты квестов лицевой стороной вниз и раздайте каждому игроку по 2 карты случайным образом. Лишние карты квестов уберите в коробку.", "metadata": {"section": "Подготовка", "subsection": "Начальная настройка"}},
{"id": "setup_005", "content": "Каждый игрок выбирает из своих карт квестов одну и кладёт её перед собой лицевой стороной вниз, не показывая остальным. Это личные квесты игроков.\nПодсказка: вашему пёсику будет проще победить монстра, если у него уже есть подходящее оружие.\nВторую карту квеста игрок кладёт лицевой стороной вверх на стол, формируя ряд общих квестов. Их может выполнить любой игрок.", "metadata": {"section": "Подготовка", "subsection": "Квесты"}},
{"id": "setup_006", "content": "Найдите среди карт комнат двустороннюю стартовую комнату. Одна сторона карты предназначена для игры втроём и вчетвером, другая — для игры вдвоём. Положите карту в центре стола нужной стороной вверх — отсюда вы начнёте свой путь по подземелью.\nСовет для опытных пёсиков: хотите бросить себе настоящий вызов? Попробуйте при игре втроём и вчетвером использовать сторону карты для игры вдвоём. Меньше выходов — сложнее приключения!", "metadata": {"section": "Подготовка", "subsection": "Стартовая комната"}},
{"id": "setup_007", "content": "Положите 3 жетона раундов на карту стартовой комнаты: сначала жетон с цифрой 3, на него жетон с цифрой 2, на самый верх жетон с цифрой 1. Это счётчик раундов.\nОстальные карты комнат перемешайте и положите на стол стопкой лицевой стороной вниз.\nПеремешайте все карты зелий лицевой стороной вниз и раздайте каждому игроку по одной случайной карте. Положите карту зелья рядом со своим планшетом лицевой стороной вверх. Лишние карты зелий положите в запас лицевой стороной вверх рядом со стопкой карт комнат.", "metadata": {"section": "Подготовка", "subsection": "Завершение настройки"}},
{"id": "setup_008", "content": "Совет для опытных пёсиков: ещё один способ сделать приключения посложнее — убрать в коробку карты зелий и играть без них.\nИзучите компоненты игрока на следующей странице — и можно начинать играть. Первым ходит игрок, который недавно гладил пёсика, — дайте ему жетон хорошего мальчика (или хорошей девочки).", "metadata": {"section": "Подготовка", "subsection": "Советы и начало игры"}},
{"id": "player_components_009", "content": "Компоненты игрока\nИмя и класс пёсика.\nСпособность пёсика. Можно применить только в свой ход. Детальные пояснения к способностям пёсиков смотрите на стр. 23–24.\nБазовые символы атаки. Если символы на карте монстра совпадают с вашими символами атаки (как базовыми, так и трофейными), то вы можете победить этого монстра.\nБазовые символы защиты. Если вы хотите пройти комнату с монстром без сражения, то вы получаете указанный на карте монстра урон. Если сумма урона на всём вашем пути равна количеству ваших символов защиты (как базовых, так и трофейных) или превышает его, то вы не можете выйти из подземелья — выберите другой маршрут или действие «Лечь спать».", "metadata": {"section": "Компоненты игрока", "subsection": "Описание планшета"}},
{"id": "player_components_010", "content": "Трофейные символы атаки и защиты (трофеи) можно получить в подземелье, побеждая монстров и собирая артефакты. Также трофейные символы приносит карта бродяги.\nЖетоны лапок лежат на планшете и используются в двух случаях:\nПри выходе из подземелья вы разыгрываете эффекты карт комнат. На картах можно оставлять жетоны лапок, чтобы в конце раунда забрать эти карты себе (подробнее в разделах «Ход игры» и «Эффекты комнат»). Вы не обязаны класть жетоны лапок на каждую карту по пути из подземелья — делайте выбор, исходя из вашего количества жетонов и вашей стратегии. Вы также не обязаны тратить все жетоны.\nДля применения способности, если вы играете за Бонька Могучего или Жанну Гавк. Подробнее на стр. 23–24.\nВ конце раунда потраченные жетоны возвращаются игрокам.", "metadata": {"section": "Компоненты игрока", "subsection": "Трофеи и жетоны лапок"}},
{"id": "player_components_011", "content": "Дополнительные жетоны можно получить с помощью карты бродяги или действия «Лечь спать». У одного игрока может быть максимум 7 жетонов на планшете.\nКарта зелья — это одноразовый дополнительный символ атаки. Вы можете использовать его, чтобы победить монстра или обезвредить ловушку. Карты зелия также помогают выполнить квесты «Стальная хватка», «Огонь в сердце» и «В яблочко». После использования уберите карту зелья в запас зелий лицевой стороной вверх. Новые карты зелия можно получить в подземелье.", "metadata": {"section": "Компоненты игрока", "subsection": "Дополнительные жетоны и зелья"}},
{"id": "player_components_012", "content": "Карта квеста. Выполненные квесты приносят монеты в конце игры. Вы можете выполнить свой личный квест и сколько угодно общих. Чтобы выполнить квест, нужно либо победить указанного монстра (можно победить несколько одинаковых монстров), либо собрать больше всех указанных трофейных символов (базовые не считаются).", "metadata": {"section": "Компоненты игрока", "subsection": "Квесты"}},
{"id": "gameplay_013", "content": "Ход игры\nИгроки ходят по очереди по часовой стрелке, начиная с первого игрока. В свой ход вы можете выполнить одно из трёх действий:\n🐾 Исследовать подземелье.\n🐾 Выйти из подземелья.\n🐾 Лечь спать.\nЗадача каждого игрока — построить наиболее выгодный для себя путь из подземелья и успеть пройти его, разыгрывая эффекты карт комнат, пока это не сделали другие игроки.", "metadata": {"section": "Ход игры", "subsection": "Общие правила"}},
{"id": "action_014", "content": "🐾 ИССЛЕДОВАТЬ ПОДЗЕМЕЛЬЕ\nВозьмите верхнюю карту в стопке комнат, изучите её и выложите на стол лицевой стороной вверх по следующим правилам:\nХотя бы один выход на карте должен совпадать с выходом на другой выложенной карте. При этом необязательно, чтобы все выходы совпадали: какие-то могут упираться в край другой карты (в стену комнаты) или вести на стол. Ведущие на стол выходы считаются выходами из подземелья. Цветная сторона карты с трофейными символами считается стеной.\nКарту можно вращать перед выкладыванием.", "metadata": {"section": "Действия", "subsection": "Исследовать подземелье"}},
{"id": "action_015", "content": "Если стопка карт комнат закончилась, исследовать подземелье нельзя — выберите другое действие.", "metadata": {"section": "Действия", "subsection": "Исследовать подземелье - ограничения"}},
{"id": "action_016", "content": "🐾 ВЫЙТИ ИЗ ПОДЗЕМЕЛЬЯ\nВыберите путь из подземелья и пройдите его по следующим правилам:\nВаш путь должен начинаться с карты стартовой комнаты и заканчиваться картой комнаты с выходом из подземелья (выходом, ведущим на стол). Количество карт комнат на пути не ограничено.\nОдну карту комнаты можно пройти и разыграть только один раз. Это касается всех карт комнат, включая стартовую комнату и порталы.\nВы должны разыгрывать эффекты карт комнат по очереди в том же порядке, в котором вы их проходите. На некоторых картах комнат вы можете оставить жетон лапки, чтобы забрать эти карты в конце раунда. Подробнее в разделе «Эффекты комнат».", "metadata": {"section": "Действия", "subsection": "Выйти из подземелья"}},
{"id": "action_017", "content": "Если на карте уже лежит жетон соперника, то вы не можете пройти эту комнату.\nВы можете использовать в этом же раунде трофеи на картах с вашими жетонами лапок: учитывайте их на своём дальнейшем пути, если будете сражаться с монстрами или обезвреживать ловушки. Если вы получили жетон лапки от бродяги, вы также можете использовать его в этом же раунде. Однако забирать карты можно только в конце раунда.\nПодсказка: помните про личные и общие квесты. Вы будете выполнять их в конце игры, а до тех пор можете планировать свой путь из подземелья таким образом, чтобы собрать как можно больше карт с нужными монстрами и трофеями.", "metadata": {"section": "Действия", "subsection": "Выйти из подземелья - особенности"}},
{"id": "action_018", "content": "После выхода из подземелья вы не участвуете в раунде и ждёте, когда выйдут остальные игроки.\nКак только из подземелья выходит первый за раунд игрок, для остальных начинается обратный отсчёт: возьмите с верха стопки комнат по 4 карты на каждого оставшегося в подземелье игрока и положите их отдельной стопкой лицевой стороной вниз.\n1 оставшийся игрок - 4 карты\n2 оставшихся игрока - 8 карт\n3 оставшихся игрока - 12 карт\nИгроки берут карты комнат с верха только этой стопки. Как только она заканчивается, исследовать подземелье нельзя — выберите другое действие.", "metadata": {"section": "Действия", "subsection": "Выйти из подземелья - обратный отсчет"}},
{"id": "action_019", "content": "ЛЕЧЬ СПАТЬ\nВозьмите дополнительный жетон лапки и положите на свой планшет — он остаётся у вас до конца игры. После этого вы не участвуете в раунде и ждёте, когда выйдут остальные игроки. Если дополнительного жетона нет, то вы ложитесь спать, не получая его.\nПодсказка: это действие можно выполнить, если вы оказались в безвыходной ситуации в подземелье.", "metadata": {"section": "Действия", "subsection": "Лечь спать"}},
{"id": "room_effects_020", "content": "Эффекты комнат\nМОНСТРЫ\nВы можете сразиться с монстром или пройти мимо него, получив урон.\nЕсли символы на карте монстра совпадают с вашими символами атаки (считаются и базовые, и трофейные символы, а также зелья), то вы побеждаете монстра и обязаны положить на карту жетон лапки.\nПобеждённый монстр приносит трофейные символы атаки или защиты, которые можно использовать в этом же раунде. Некоторые монстры вдобавок к трофеям приносят монеты в конце игры. Дракон приносит 4 монеты вместо трофеев. Стрекоключ приносит трофейный ключ — с ним вы можете проходить через все двери в подземелье и открыть в конце игры один сундук.", "metadata": {"section": "Эффекты комнат", "subsection": "Монстры"}},
{"id": "room_effects_021", "content": "Если вы не можете или не хотите сражаться с монстром, но вам нужно пройти комнату, то вы получаете указанный на карте монстра урон, вычитая его из общего количества ваших символов защиты (и базовых, и трофейных). Если сумма урона за весь путь равна количеству символов вашей защиты или превышает его, то вы не можете идти этим путём — выберите другой или ложитесь спать.\nВы можете заранее перед выходом из подземелья мысленно пройти свой путь и посчитать, сколько символов защиты вы можете потерять.", "metadata": {"section": "Эффекты комнат", "subsection": "Монстры - урон"}},
{"id": "room_effects_022", "content": "Карта монстра\nНазвание монстра.\nСимволы атаки, необходимые для победы.\nУрон.\nТрофей.\nМонета.\nПример сражений\nДана играет за Робин Гудгёрл и проходит комнату с мимиком. Для победы над мимиком нужен один лук и один огненный шар. Базовых символов атаки Робин хватает, чтобы победить мимика (трофейных у неё пока нет). Дана кладёт на карту монстра жетон лапки и продолжает свой путь по подземелью.\nВ следующей комнате она сталкивается с цербером. Для победы над ним требуется 3 меча, но у Даны всего 1 меч — трофей от побеждённого в предыдущей комнате мимика. Этого недостаточно, и Дана теряет 2 защиты. У неё остаётся всего 1 защита, так что на последующем пути из подземелья Дана избегает сражений с монстрами, которые ей не под силу.", "metadata": {"section": "Эффекты комнат", "subsection": "Монстры - примеры"}},
{"id": "room_effects_023", "content": "🐾 МОНЕТЫ И АРТЕФАКТЫ\nВы можете положить на карту жетон лапки, чтобы забрать эту карту в конце раунда. Артефакт приносит два трофейных символа атаки. Всего в игре три разных артефакта. Монеты нужны для победы.\nВ игре есть комнаты с одной и с двумя монетами.", "metadata": {"section": "Эффекты комнат", "subsection": "Монеты и артефакты"}},
{"id": "room_effects_024", "content": "🐾 ЛОВУШКИ\nЧтобы пройти ловушку, вы обязаны либо обезвредить её, либо положить жетон лапки и взять эту карту в конце раунда по обычным правилам.\nЧтобы обезвредить ловушку, у вас должна быть карта комнаты (включая бродягу) или зелья с подходящим трофеем (базовые символы не обезвреживают ловушку). Это может быть карта, которую вы получили в предыдущих раундах или разыграли ранее в этом раунде.\n🛡 щит\n⚔ меч\n🏹 лук\n🔥 огненный шар", "metadata": {"section": "Эффекты комнат", "subsection": "Ловушки"}},
{"id": "room_effects_025", "content": "Положите карту с трофеем лицевой стороной вниз поверх ловушки — теперь она обезврежена, а трофей утерян и не может использоваться. Другие игроки не могут пройти по обезвреженной ловушке. На обезвреженную ловушку не нужно класть жетон лапки.\nЕсли вы обезвредили ловушку картой, разыгранной в этом раунде, то в подземелье может образоваться пустое пространство с жетоном лапки (см. пример на стр. 15). Другие игроки не могут его пройти или заполнить новой картой комнаты.\nВ конце раунда уберите в коробку обе карты — карту ловушки и карту, которая её обезвредила (кроме зелья — его положите в запас зелий). Больше они не появятся в игре.", "metadata": {"section": "Эффекты комнат", "subsection": "Ловушки - обезвреживание"}},
{"id": "room_effects_026", "content": "Если вам нечем обезвредить ловушку или вы не хотите этого делать, положите на карту жетон лапки. В конце игры она отнимет одну монету в качестве штрафа.\nКарта ловушки\nТрофей, необходимый для обезвреживания ловушки.\nШтраф.\nСимвол немедленного эффекта.", "metadata": {"section": "Эффекты комнат", "subsection": "Ловушки - штрафы"}},
{"id": "room_effects_027", "content": "⚡ Символ немедленного эффекта встречается на ловушках и комнатах зелий и означает, что для разыгрывания карты необходимо немедленно потратить трофей (если вы обезвреживаете ловушку) или взять карту зелья из запаса.\nПример обезвреживания ловушки\nЛена играет за Жанну Гавк. В предыдущей комнате она победила зомби-белку и оставила на карте жетон лапки. Теперь ей нужно пройти ловушку. Для обезвреживания требуется трофейный меч. Лена решает использовать меч от побеждённой только что зомби-белки: она переворачивает карту зомби-белки и кладёт её поверх ловушки. Там, где лежала карта монстра, остаётся жетон лапки. Лена лишилась трофейного меча, зато не придётся платить штраф в конце игры.", "metadata": {"section": "Эффекты комнат", "subsection": "Ловушки - примеры"}},
{"id": "room_effects_028", "content": "🐾 БРОДЯГА\nВы можете положить на карту жетон лапки, чтобы нанять бродягу — он будет помогать вам до конца игры, но ему потребуется заплатить.\nСразу выберите одну из двух сторон с трофейными символами — их вы будете использовать до конца игры без возможности поменять сторону.\nСторона с двумя трофеями приносит два трофея, однако в каждом сражении с монстром вы можете использовать только один из них на ваш выбор. Их также можно использовать, чтобы обезвредить ловушку (тогда платить бродяге не придётся).", "metadata": {"section": "Эффекты комнат", "subsection": "Бродяга"}},
{"id": "room_effects_029", "content": "Сторона с одним трофеем и лапкой приносит один трофей и дополнительный жетон лапки — сразу возьмите его и положите на свой планшет. Им можно воспользоваться в этом же раунде. Если вы выбрали эту сторону и обезвредили картой бродяги ловушку, уберите со своего планшета один жетон лапки — он становится дополнительным и не возвращается на планшет в конце раунда. Если вы уже потратили все жетоны в этом раунде, включая полученный от бродяги, то уберите один жетон с планшета перед началом следующего раунда.\nПри подсчёте монет в конце игры отнимите одну монету за помощь бродяги. Если вы обезвредили бродягой ловушку, то отнимать монету не нужно.\nЕсли вам не нужен бродяга, пройдите эту комнату, не оставляя на ней жетон лапки.", "metadata": {"section": "Эффекты комнат", "subsection": "Бродяга - особенности"}},
{"id": "room_effects_030", "content": "СУНДУК\nВы можете положить на карту жетон лапки, чтобы забрать сундук. В конце игры сундук принесёт 4 монеты, но только в том случае, если у вас есть ключ.\nПолучить ключ можно, победив стрекоключа. У Плохиша Плуто есть ключ с начала игры. Одним ключом можно открыть только один сундук. У одного игрока может быть несколько ключей и сундуков.", "metadata": {"section": "Эффекты комнат", "subsection": "Сундук"}},
{"id": "room_effects_031", "content": "ЗЕЛЬЯ\nВы можете положить на карту жетон лапки и сразу взять любую карту зелья из запаса на ваш выбор. Можно взять ту карту зелья, которую убрал в запас ранее другой игрок. Саму карту комнаты вы забираете в конце раунда по общим правилам, однако в конце игры она не приносит ничего. У вас может быть сколько угодно карт зелий.", "metadata": {"section": "Эффекты комнат", "subsection": "Зелья"}},
{"id": "room_effects_032", "content": "ДВЕРЬ\nВы можете пройти через дверь, если у вас есть ключ. Боньк Могучий может применить свою способность, чтобы пробить дверь, не имея ключа. Можно использовать трофейный ключ, полученный в этом же раунде или в предыдущих раундах, или ключ Плохиша Плуто, если играете за него. Если у вас нет ключа, то выход с дверью для вас заблокирован. Ключ не сбрасывается — одним ключом можно открыть любое количество дверей в подземелье.", "metadata": {"section": "Эффекты комнат", "subsection": "Дверь"}},
{"id": "room_effects_033", "content": "ПОРТАЛ\nЕсли в подземелье больше одной карты портала, то вы можете перемещаться между ними как между картами комнат с общим выходом. На порталы не нужно класть жетон лапки.", "metadata": {"section": "Эффекты комнат", "subsection": "Портал"}},
{"id": "round_end_034", "content": "Конец раунда\nРаунд заканчивается, когда все игроки вышли из подземелья или легли спать. Игрок, который сделал последний ход в раунде, сразу забирает себе жетон хорошего мальчика (или хорошей девочки). Затем каждый игрок забирает себе те карты комнат, на которых лежат его жетоны лапок, а сами жетоны возвращает на свой планшет.\nКарты монет, сундуков, необезвреженных ловушек положите отдельно рядом с собой лицевой стороной вниз.\nКарты побеждённых монстров, артефактов и бродяг положите слева под свой планшет как на иллюстрации. Трофейный ключ положите таким же образом под планшет справа. Бродягу положите той стороной, которую вы выбрали, когда наняли его.", "metadata": {"section": "Конец раунда", "subsection": "Процедура"}},
{"id": "round_end_035", "content": "Уберите в коробку карты обезвреженных ловушек вместе с лежащими на них картами (кроме карт зелий — их уберите в запас зелий).\nСнимите верхний жетон со счётчика раундов. Если это был последний жетон, игра сразу заканчивается. Если нет, то замешайте оставшиеся в подземелье карты комнат (кроме стартовой) обратно в стопку со всеми картами и начните новый раунд по тем правилам. Пёсики снова оказываются в подземелье, но уже более подготовленные к новым испытаниям.", "metadata": {"section": "Конец раунда", "subsection": "Подготовка к новому раунду"}},
{"id": "game_end_036", "content": "Конец игры и подсчёт монет\nПосле третьего раунда игра заканчивается.\nСложите и посчитайте количество золотых монет на всех своих картах комнат. Помните, что сундук приносит монеты только в том случае, если у вас есть ключ, и один ключ открывает только один сундук.\nОтнимите монеты за карты ловушек и бродяг, если они у вас есть.\nПолучите монеты за выполненные личные и общие квесты. Монеты за личный квест может получить только тот игрок, которому принадлежит эта карта квеста. Монеты за общий квест может получить любой игрок, который его выполнил (это могут быть несколько игроков).", "metadata": {"section": "Конец игры", "subsection": "Подсчет монет"}},
{"id": "game_end_037", "content": "Квесты с монстрами приносят монеты за каждого указанного побеждённого монстра — например, если вы победили двух ифритов, то считается, что вы дважды выполнили квест «Последнее желание» и получаете 4 монеты.\nПримечание: квест с трофеями («Стальная хватка», «Огонь в сердце», «В яблочко») может выполнить только один игрок, даже если квест общий. Если несколько игроков набрали одинаковое количество трофейных символов, то ничья разрешается в пользу игрока, у которого больше базовых указанных символов. Если ничья сохраняется, то считается, что никто не выполнил квест.", "metadata": {"section": "Конец игры", "subsection": "Квесты"}},
{"id": "game_end_038", "content": "Победителем становится игрок, который набрал больше всех монет.\nПри ничьей побеждает тот, кто выполнил больше квестов — как личных, так и общих. Если ничья сохраняется, то участники ничьей делят победу и радостно виляют хвостиками. Гав-гав!", "metadata": {"section": "Конец игры", "subsection": "Определение победителя"}},
{"id": "example_039", "content": "Пример первого раунда\nИграют Лена, Дана, Луиза и Матвей. Луиза (➡️) играет за Плохиша Плуто и первой решает выйти из подземелья. Начиная со стартовой комнаты, она проходит желатинового кота и побеждает его с помощью базовых символов атаки 🐾, затем проходит гигаклеща и также побеждает его 🐾, после чего выходит из подземелья (карта ловушки была выложена рядом с гигаклещом уже после того, как Луиза закончила свой ход).\nЛена, Дана и Матвей отсчитывают 12 карт комнат с верха стопки и продолжают исследовать подземелье.", "metadata": {"section": "Примеры", "subsection": "Пример первого раунда - начало"}},
{"id": "example_040", "content": "Следующей из подземелья выходит Лена (➡️), она играет за Жанну Гавк: сначала Лена побеждает ифрита 🔥, затем проходит комнату с монетой 🪙, затем побеждает гигаклеща, используя зелье ловкости вдобавок к своему базовому луку 🏹.\nСледующей выходит Дана (➡️), она играет за Робин Гудгёрл. Дана выбирает путь с тремя монетами и без монстров 🐉🐉🐉. Комнату с порталом она проходит без разыгрывания эффекта, поскольку других порталов на поле нет.", "metadata": {"section": "Примеры", "subsection": "Пример первого раунда - продолжение"}},
{"id": "example_041", "content": "Последним из подземелья выходит Матвей (➡️), он играет за Шарика Огненного. Сначала он нанимает бродягу 🐾 и решает использовать сторону с огненным шаром и лапкой. Матвей сразу берёт дополнительный жетон лапки и продолжает свой путь: проходит сундук 🧰, побеждает шоколадного голема, используя два базовых огненных шара и трофейный огненный шар от бродяги 🔥, побеждает мимика, используя базовый лук и базовый огненный шар 🔥 и выходит из подземелья. После этого Матвей сразу забирает жетон хорошего мальчика, поскольку он последним сделал ход в этом раунде.", "metadata": {"section": "Примеры", "subsection": "Пример первого раунда - завершение"}},
{"id": "example_042", "content": "Игроки разбирают карты: у Луизы 2 трофея; у Лены 2 трофея и 1 монета; у Даны 3 монеты; у Матвея 3 трофея (от голема, бродяги и мимика), 2 монеты и сундук — для него понадобится ключ, который Матвей надеется добыть в следующих раундах.", "metadata": {"section": "Примеры", "subsection": "Пример первого раунда - итоги"}},
{"id": "example_043", "content": "Пример подсчёта монет\nОбщие квесты игроков:\nСоловьи с пением - 2 монеты\nПоследнее желание - 4 монеты\nНе нападай - 4 монеты\nСемь клыков - 4 монеты\nЛена подсчитывает монеты: у неё 1 карта комнаты с монетой и 4 побеждённых монстра с монетами — итого 4 монеты. Также Лена получает 4 монеты за сундук, для которого у неё есть трофейный ключ.", "metadata": {"section": "Примеры", "subsection": "Пример подсчета монет - начало"}},
{"id": "example_044", "content": "Затем Лена проверяет личный квест «Огонь в сердце»: у неё 6 трофейных символов огненного шара (от волшебника, желатинового кота, ифрита, зелья магии и бродяги). Это больше, чем у кого-либо из игроков, и Лена получает 5 монет за выполненный личный квест.\nЗатем она проверяет общие квесты: она победила двух желатиновых котов, двух ифритов и одного цербера — за каждого монстра Лена получает 2 монеты, итого 10 монет.\nВ конце она отнимает 2 монеты за помощь бродяги и ловушку, которую не смогла обезвредить.\nИтого у Лены 4+4+5+10−2=21 монета.", "metadata": {"section": "Примеры", "subsection": "Пример подсчета монет - завершение"}},
{"id": "abilities_045", "content": "Пояснения к способностям пёсиков\nБоньк Могучий\nЧтобы пробить стену или дверь, положите на неё жетон лапки. Теперь считается, что на месте жетона есть проход. Таким образом можно пробить выход из подземелья. Другие пёсики тоже могут воспользоваться пробитым проходом. Потраченный жетон служит только обозначением прохода и не позволяет забрать карту в конце раунда.", "metadata": {"section": "Способности пёсиков", "subsection": "Боньк Могучий"}},
{"id": "abilities_046", "content": "Жанна Гавк\nСпособность Жанны Гавк можно применить только при выходе из подземелья и только на того монстра, которого Жанна не может победить (кроме дракона). В общей сложности Жанна Гавк кладёт на карту монстра 2 жетона лапок: чтобы применить способность и чтобы забрать карту по общим правилам.\nПример: Жанна Гавк не может победить шоколадного голема: она кладёт один жетон лапки, чтобы применить свою способность, и второй чтобы обозначить победу над монстром.", "metadata": {"section": "Способности пёсиков", "subsection": "Жанна Гавк"}},
{"id": "abilities_047", "content": "Плохиш Плуто\nКлюч Плохиша работает так же, как и трофейный ключ: он открывает любые выходы с дверями в подземелье и только один сундук в конце игры.\nРобин Гудгёрл\nЛовушки, которые игнорирует Робин, не считаются обезвреженными (не уходят из игры). Робин не может использовать трофеи и бродягу для обезвреживания ловушек, даже если захочет.", "metadata": {"section": "Способности пёсиков", "subsection": "Плохиш Плуто и Робин Гудгёрл"}},
{"id": "abilities_048", "content": "Рута Снежная\nУ Руты 5 символов защиты, все они считаются базовыми.\nШарик Огненный\nНа способность Шарика не тратится жетон лапки. Шарик может поворачивать карты в любую сторону (даже нарушая правила выкладывания карт), но не может переворачивать их лицевой стороной вниз. Картами вокруг стартовой комнаты считаются те четыре карты, которые соприкасаются со сторонами стартовой комнаты.", "metadata": {"section": "Способности пёсиков", "subsection": "Рута Снежная и Шарик Огненный"}},
{"id": "credits_049", "content": "Создатели игры\nАвтор игры: Денис Сайдашев \nХудожник персонажей: Никита Орлов \nХудожники: uildrim, Екатерина Жарова, Кристина Соозар \nРазвитие игры: Екатерина Рейес \nПродюсер: Владимир Грачёв \nАрт-директор: Сергей Дулин \nДизайн и вёрстка: Кристина Соозар \nВыпускающий редактор: Анна Давыдова \nДиректор издательского департамента: Александр Киселев \nИгру тестировали: Елена Ворноскова, Луиза Кретова, Дана Кузнецова, Матвей Чистяков и другие", "metadata": {"section": "Кредиты", "subsection": "Создатели"}},
{"id": "credits_050", "content": "Издатель: ООО «Мир Хобби»\nОбщее руководство: Михаил Акулов Руководство производством: Иван Попов Главный редактор: Валентин Матюша Главный дизайнер-верстальщик: Иван Суховей Корректор: Ольга Португалова Креативный директор: Николай Пегасов Международное продвижение: Мария Никольская\nЕсли вы придумали настольную игру и желаете, чтобы она была издана, пишите на newgame@hobbyworld.ru\nОсобая благодарность выражается Илье Карпинскому.\nПерепечатка и публикация правил, компонентов и иллюстраций игры без разрешения правообладателя запрещены.\n© 2024 ООО «Мир Хобби». Все права защищены.\nВерсия правил 1.1\nhobbyworld.ru", "metadata": {"section": "Кредиты", "subsection": "Издатель и контакты"}},
]

In [3]:
class Metadata(BaseModel):
    section: str
    subsection: str

class RulePydantic(BaseModel):
    id: str
    content: str
    metadata: Metadata


In [4]:
@Entity()
class Rule:
    id = Id
    entity_id = String
    content = String
    section = String
    subsection = String
    game = String
    vector = Float32Vector(index=HnswIndex(
        dimensions=768,
        distance_type=VectorDistanceType.EUCLIDEAN
    ))

In [5]:
rules_lines = [RulePydantic(**obj) for obj in rules]

In [6]:
store = Store()
rules_box = Box(store=store, entity=Rule)

---

In [7]:
model = SentenceTransformer("google/embeddinggemma-300m").to("cpu")

In [8]:
box_objects = [Rule(
    entity_id = obj.id,
    content = obj.content,
    section = obj.metadata.section,
    subsection = obj.metadata.subsection,
    game = "Подземелье и пёсики",
    vector = model.encode(obj.content).tolist()
) for obj in tqdm(rules_lines)]

100%|██████████| 50/50 [00:13<00:00,  3.63it/s]


In [9]:
rules_box.put(box_objects)

In [10]:
rules_box.get(1)

In [11]:
store.close()